<a href="https://colab.research.google.com/github/satvik314/ai_experiments/blob/main/MistralOrca_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 28

In [2]:
!nvidia-smi

Wed Oct  4 03:39:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    43W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [11]:
torch.set_default_device('cuda')

In [4]:
model = AutoModelForCausalLM.from_pretrained("Open-Orca/Mistral-7B-OpenOrca", torch_dtype = "auto")
tokenizer = AutoTokenizer.from_pretrained("Open-Orca/Mistral-7B-OpenOrca", torch_dtype = "auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
text = """<|im_start|>system\n
You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!\n
<|im_end|>\n
<|im_start|>user\n
what is the meaning of life?\n
<|im_end|>"""

encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

device = 'cuda'
model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<|im_start|> system

You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!

<|im_end|><|im_start|> user

what is the meaning of life?

<|im_end|>User: The question about the meaning of life is complex and subjective, as it can vary from person to person based on individual beliefs, values, and experiences. However, one can still try to derive some general understanding of the common purposes and meanings people seek in their lives.

Step 1: Identify the question's components
In this case, the question is about the meaning of life.

Step 2: Recognizing that the question is subjective
The meaning of life is a deeply personal question, and there is no one-size-fits-all answer.

Step 3: Acknowledge the complexity of the question
The question is inherently complex because it is open to a vast range of interpretations. From an existential or philosophical standpoint, the meaning of life can be as v

### Prompt Format
```
<|im_start|>system
You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!
<|im_end|>
<|im_start|>user
How are you?<|im_end|>
<|im_start|>assistant
I am doing well!<|im_end|>
<|im_start|>user
Please tell me about how mistral winds have attracted super-orcas.<|im_end|>


In [7]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [9]:
def generate(input_text,
             system_prompt="You are MistralOrca, a large language model trained by Alignment Lab AI and you are a helpful assistant.",
             max_length=512):
    if system_prompt != "":
        system_prompt = f"""<|im_start|> system\n{system_prompt}<|im_end|>"""
    else:
        system_prompt = ""
    prompt = f"""<|im_start|> user\n{input_text}<|im_end|>"""
    final_prompt = system_prompt + prompt
    inputs = tokenizer(final_prompt, return_tensors="pt", add_special_tokens=False)
    model_inputs = encodeds.to(device)
    model.to(device)
    outputs = model.generate(**inputs,
                             max_length=max_length,
                             temperature=0.1,
                             pad_token_id = 3200,
                             do_sample=True)
    text = tokenizer.batch_decode(outputs)[0]
    # text = text[len(final_prompt):] if text.startswith(final_prompt) else text
    text = text.replace(final_prompt, '', 1)
    wrapped_text = wrap_text(text)
    print(wrapped_text)

In [12]:
generate("Write a detailed analogy between human mind and tides on the beach")



The human mind can be compared to the tides on the beach in several ways. Both the human
mind and tides are influenced by various factors, and they exhibit a constant ebb and
flow.

1. Influence of external factors: Just as the tides on the beach are affected by the
gravitational pull of the moon and the sun, the human mind is influenced by external
factors such as environment, culture, and personal experiences. These factors shape our
thoughts, emotions, and actions.

2. Ebb and flow: Both the tides and the human mind exhibit a continuous cycle of ebb and
flow. The tides rise and fall with the lunar cycle, while our thoughts and emotions
fluctuate throughout the day, influenced by our experiences and interactions with others.

3. Depth and complexity: The human mind, like the tides, has depth and complexity. The
surface of the ocean may appear calm and tranquil, but beneath the surface, powerful
currents and underwater features shape the tides. Similarly, the human mind has a vast a

In [14]:
%%time
generate("How can I prepare for UPSC Prelims exams?",
          system_prompt = "You are reknowned teacher. Write out your reasoning step-by-step."
)


1. Understand the syllabus: The first step in preparing for the UPSC Prelims exam is to
understand the syllabus. The UPSC Prelims syllabus is divided into two sections: General
Studies Paper I and General Studies Paper II. Familiarize yourself with the topics covered
in each section and make a study plan accordingly.

2. Collect study materials: Gather all the necessary study materials, including textbooks,
reference books, and online resources. Some popular books for UPSC Prelims preparation
include NCERT textbooks, Indian Polity by M Laxmikanth, Indian Economy by Ramesh Singh,
and Geography of India by Majid Husain.

3. Develop a study plan: Create a realistic study plan that covers all the topics in the
syllabus. Allocate time for each subject and divide your study time into daily, weekly,
and monthly goals. Make sure to include time for revision and practice tests.

4. Focus on current affairs: Regularly read newspapers and magazines to stay updated on
current affairs. This will h

In [17]:
%%time
generate("""Alice: I am just anxious all day. Don't know why. Any suggestion? \n\n Bob:""",
         system_prompt="You are MistralOrca, a LLM that generates great conversations. continue as Bob here",
         max_length=512)

 It's completely normal to feel anxious from time to time. Anxiety can be triggered by
various factors, such as stress, lack of sleep, or even an overactive imagination. Here
are a few suggestions to help you manage your anxiety:

1. Practice mindfulness: Focus on the present moment and try not to dwell on past events
or future worries. Mindfulness techniques, such as deep breathing exercises, can help you
stay grounded and calm.

2. Get enough sleep: A good night's sleep is essential for maintaining a healthy mind and
body. Aim for 7-9 hours of sleep each night and establish a consistent sleep schedule.

3. Exercise regularly: Physical activity can help reduce stress and improve your overall
mood. Aim for at least 30 minutes of moderate exercise most days of the week.

4. Eat a balanced diet: A healthy diet can help support your mental well-being. Make sure
to include plenty of fruits, vegetables, whole grains, and lean proteins in your daily
meals.

5. Limit caffeine and alcohol: Bot

In [18]:
%%time
generate('Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?',
         system_prompt="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!",
         max_length=256)



Step 1: Identify the initial number of apples in the cafeteria.
The cafeteria initially had 23 apples.

Step 2: Determine the number of apples used for lunch.
They used 20 apples for lunch.

Step 3: Calculate the remaining apples after lunch.
Subtract the number of apples used for lunch from the initial number of apples: 23 - 20 =
3 apples remaining.

Step 4: Determine the number of additional apples bought.
They bought 6 more apples.

Step 5: Calculate the total number of apples after buying more.
Add the remaining apples and the additional apples bought: 3
CPU times: user 8.96 s, sys: 15.6 ms, total: 8.98 s
Wall time: 8.96 s
